<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#고객-등급-분류" data-toc-modified-id="고객-등급-분류-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>고객 등급 분류</a></span></li><li><span><a href="#사용할-리텐션-마케팅-제안-함수-코드-부분-(Churn-Value-제외-버전)" data-toc-modified-id="사용할-리텐션-마케팅-제안-함수-코드-부분-(Churn-Value-제외-버전)-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>사용할 리텐션 마케팅 제안 함수 코드 부분 (Churn Value 제외 버전)</a></span><ul class="toc-item"><li><span><a href="#기업/개인-고객-분류-+-혜택-제안-적용되서-나오게-해보기" data-toc-modified-id="기업/개인-고객-분류-+-혜택-제안-적용되서-나오게-해보기-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>기업/개인 고객 분류 + 혜택 제안 적용되서 나오게 해보기</a></span></li><li><span><a href="#사용할-리텐션-마케팅-제안-함수-코드-부분-(Churn-Value-제외-버전)" data-toc-modified-id="사용할-리텐션-마케팅-제안-함수-코드-부분-(Churn-Value-제외-버전)-2.2"><span class="toc-item-num">2.2&nbsp;&nbsp;</span>사용할 리텐션 마케팅 제안 함수 코드 부분 (Churn Value 제외 버전)</a></span></li></ul></li></ul></div>

In [14]:
import numpy as np
import pandas as pd
import plotly.offline as py # visualization
py.init_notebook_mode(connected=True) # visualization
import plotly.graph_objs as go # visualization
from plotly.subplots import make_subplots
import plotly.figure_factory as ff # visualization
import plotly.express as px 
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot 
import cufflinks as cf 
import warnings

In [15]:
warnings.filterwarnings('ignore')
cf.go_offline() 
init_notebook_mode(connected=True)

In [16]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity="all"

In [17]:
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

# Graphics in retina format are more sharp and legible
%config InlineBackend.figure_format = 'retina'

In [18]:
# unicode minus를 사용하지 않기 위한 설정 (minus 깨짐현상 방지)
plt.rcParams['axes.unicode_minus'] = False
# 한글 텍스트를 제대로 표시하기 위해 'font.family' 사용
plt.rcParams['font.family'] = 'AppleGothic'

In [19]:
# csv 파일 불러오기
df = pd.read_csv('./data/Churn.csv')

In [20]:
df.head()

,Customer ID,Age,Number of Dependents,Membership,Satisfaction Score,Tech services,Streaming services,Combined Product,Contract,Tenure in Months,Monthly Charge,Total Revenue,Churn Value,CLTV,Total Charges
0,8779-QRDMV,78,0,NaN,3,1,1,2,Month-to-Month,1,39.65,59.65,1,5433,39.65
1,7495-OOKFY,74,1,Offer E,3,1,0,1,Month-to-Month,8,80.65,1024.10,1,5302,633.30
2,1658-BYGOY,71,3,Offer D,2,0,2,1,Month-to-Month,18,95.45,1910.88,1,3179,1752.55
3,4598-XLKNJ,78,1,Offer C,2,2,2,1,Month-to-Month,25,98.50,2995.07,1,5337,2514.50
4,4846-WHAFZ,80,1,Offer C,2,0,0,1,Month-to-Month,37,76.50,3102.36,1,2793,2868.15


In [21]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7043 entries, 0 to 7042
Data columns (total 15 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   Customer ID           7043 non-null   object 
 1   Age                   7043 non-null   int64  
 2   Number of Dependents  7043 non-null   int64  
 3   Membership            3166 non-null   object 
 4   Satisfaction Score    7043 non-null   int64  
 5   Tech services         7043 non-null   int64  
 6   Streaming services    7043 non-null   int64  
 7   Combined Product      7043 non-null   int64  
 8   Contract              7043 non-null   object 
 9   Tenure in Months      7043 non-null   int64  
 10  Monthly Charge        7043 non-null   float64
 11  Total Revenue         7043 non-null   float64
 12  Churn Value           7043 non-null   int64  
 13  CLTV                  7043 non-null   int64  
 14  Total Charges         7043 non-null   float64
dtypes: float64(3), int64(

In [22]:
# 'Membership' 컬럼의 결측치를 "None"으로 대체
df['Membership'].fillna('None', inplace=True)

In [27]:
df['CLTV'].sort_values().max()
df['CLTV'].sort_values().min()

6500

2003

## 고객 등급 분류 

    CLTV
    * 2000-4000: 저가치 고객
    * 4001-6000: 잠재 이탈율이 큰 고객으로 우리가 눈여겨봐야할 고객층
    * 6000 이상: 고가치 고객

## 사용할 리텐션 마케팅 제안 함수 코드 부분 (Churn Value 제외 버전)

    * 기업/개인 고객 분류 + 혜택 제안 적용되서 나오게 해보기

In [28]:
# input 함수는 주석 처리 해놓음.
# 컬럼 설명 및 조건별 마케팅 제안 설명 B2B/B2C 로 나누어서 해놨는데 만약 잘못된 부분 있으면 알려주세요.

def get_cltv_description(CLTV, total_revenue, tenure_months, num_dependents):
    """
    ['CLTV']: Amount of total expected value per person that might contribute in the future.
    ['total_revenue']: The total annual earnings from each customer.
    ['tenure_months']: The duration for which each customer has retained their contracts.
    ['num_dependent']: The number of children in family customers.
    
   # Loyalty Class Customers(CLTV 높은 고가치 고객층)
   
   (B2B)
    - CLTV above 6000, total_revenue above $10,000
    Marketing Suggestions:  '결합 할인 + 요금할인(20%)' 제안 가능
    - CLTV between 5000 and 6000, total_revenue above $10,000
    Marketing Suggestions: '결합 할인 + 요금할인(18%)' 제안 가능
    - CLTV between 4000 and 5000, total_revenue between $6,000 and $10,000
    Marketing Suggestions: '결합 할인 + 요금할인(15%)' 제안 가능
    
    (B2C)
    - CLTV above 5000, tenure_months at least above 60 months, num_dependets above 0
    Marketing Suggestions: '결합 할인 + 요금할인(10%)' 제안 가능
    
    
    # Middle Class Customers (Potential customers who might churn)
    
    (B2B)
    - CLTV between 5000 and 6000, total_revenue between $4,500 and $6,000
    Marketing Suggestions: '요금 할인(13%)' 제안 가능
    - CLTV between 4001 and 5000, total_revenue between $2,000 and $4,500
    Marketing Suggestions: '요금 할인(10%)' 제안 가능
    
    (B2C)
    - CLTV between 4000 and 6000, total_revenue tenure_months between 36months and 60months
    Marketing Suggestions: '할인 혜택 or 업그레이드 or 맞춤형' 제안 가능
    
    
    # Lower Class Customer, including exceptions
    
    (B2B) => Business clients who have lower CLTV, but unexpectedly higher total revenue.
    - CLTV between 2000 and 4000, total_revenue above $6,000
    Marketing Suggestions: '요금 할인(15%)' 제안 가능
    - CLTV between 2000 and 4000, total_revenue between $4,000 and $6,000
    Marketing Suggestions: '요금 할인(10%)' 제안 가능
    
    (B2C)
    - CLTV below 4000, tenure_months between 24months and 36months
    Marketing Suggestions: '할인 혜택 or 업그레이드 or 맞춤형' 제안 가능
    - CLTV below 4000, tenure_months less than 24months
    Marketing Suggestions: '업그레이드 or 맞춤형'제안 가능
    # 경우의 수 추가 부분 
     - CLTV above 4000, tenure_months less than 24months
    Marketing Suggestions: '업그레이드 or 맞춤형'제안 가능
    - CLTV between 2000 and 4000, tenure_months less than 24months
     Marketing Suggestions: '제안할 마케팅 없음'

    """
    
    if CLTV >= 6000 and total_revenue >= 10000:
        return "CLTV 높은 고가치 기업 고객님 입니다. [결합 할인 + 요금할인(20%)] 제안 가능합니다."
    elif 5000 <= CLTV < 6000 and total_revenue >= 10000:
        return "CLTV 높은 고가치 기업 고객님 입니다. [결합 할인 + 요금할인(18%)] 제안 가능합니다."
    elif 4000 <= CLTV < 5000 and 6000 >= total_revenue < 10000:
        return "CLTV 높은 고가치 기업 고객님 입니다. [결합 할인 + 요금할인(15%)] 제안 가능합니다."
    elif CLTV >= 5000 and tenure_months >= 60 and num_dependents > 0:
        return "CLTV 높은 고가치 개인 고객님 입니다. [결합 할인 + 요금할인(10%)] 제안 가능합니다."
    elif 5000 <= CLTV < 6000 and 4500 <= total_revenue < 6000:
        return "CLTV 중간 가치 기업 고객님(잠재적 이탈 고객층) 입니다. [요금 할인(13%)] 제안 가능합니다."
    elif 4001 <= CLTV < 5000 and 2000 <= total_revenue < 4500:
        return "CLTV 중간 가치 기업 고객님(잠재적 이탈 고객층) 입니다. [요금 할인(10%)] 제안 가능합니다."
    elif 4000 <= CLTV < 6000 and 36 <= tenure_months < 60:
        return "CLTV 중간 가치 개인 고객님(잠재적 이탈 고객층) 입니다. [할인 혜택 or 업그레이드 or 맞춤형] 제안 가능합니다."
    elif 2000 <= CLTV < 4000 and total_revenue >= 6000:
        return "CLTV 낮지만 수익 가치가 높은 기업 고객님 입니다. [요금 할인(15%)] 제안 가능합니다."
    elif 2000 <= CLTV < 4000 and 4000 <= total_revenue < 6000:
        return "CLTV 낮지만 수익이 중간 가치인 기업 고객님 입니다. [요금 할인(10%)] 제안 가능합니다."
    elif CLTV > 4000 and 24<= tenure_months < 36:
        return "CLTV 낮은 저가치 개인 고객님 입니다. [할인 혜택 or 업그레이드 or 맞춤형] 제안 가능합니다."
    # 경우의 수 추가 부분 
    elif CLTV <= 4000 and tenure_months < 24:
        return "CLTV 낮은 저가치 개인 고객님 입니다. [업그레이드 or 맞춤형] 제안 가능합니다."
    elif 2000 <= CLTV < 4000 and tenure_months < 24:
        return "CLTV 낮은 저가치 개인 고객님 입니다. 제안할 마케팅 제안이 없습니다."        
    else:
        return "제안할 마케팅 제안이 없습니다."

def get_cltv_description_by_customer_id(df):
    try:
        customer_id = input("고객 번호를 입력하세요: ")  # 사용자로부터 고객 번호를 입력받음
        customer_data = df[df['Customer ID'] == customer_id]
        if customer_data.empty:
            return "고객 번호를 찾을 수 없습니다."
        CLTV = customer_data['CLTV'].values[0]
        total_revenue = customer_data['Total Revenue'].values[0]
        tenure_months = customer_data['Tenure in Months'].values[0]
        num_dependents = customer_data['Number of Dependents'].values[0]
        description = get_cltv_description(CLTV, total_revenue, tenure_months, num_dependents)
        return description
    except IndexError:
        return "고객 번호를 찾을 수 없습니다."

# 예시
description = get_cltv_description_by_customer_id(df)
print(description)

고객 번호를 입력하세요: 1658-BYGOY
CLTV 낮은 저가치 개인 고객님 입니다. [업그레이드 or 맞춤형] 제안 가능합니다.
